In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import re 

In [2]:
sheets_to_load = ['Germany',
                  'Italy',
                  'Poland',
                  'Switzerland',
                  'Sweden',
                  'Thailand',
                  'UK',
                  'Vietnam',
                  'Australia']

# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\SEPTEMBER\q16-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\OCTOBER\q16-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\NOVEMBER\q16-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\DECEMBER\q16-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\2025\JANUARY\q16-compilation.xlsx', sheet_name=sheets_to_load)
selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\2025\JULY\q16-compilation.xlsx', sheet_name=sheets_to_load)

Germany = selected_sheets['Germany']
Italy = selected_sheets['Italy']
Poland = selected_sheets['Poland']
Switzerland = selected_sheets['Switzerland']
Sweden = selected_sheets['Sweden']
Thailand = selected_sheets['Thailand']
UK = selected_sheets['UK']
Vietnam = selected_sheets['Vietnam']
Australia = selected_sheets['Australia']

### Germany

In [3]:
Germany.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                                                                --------------  -----  
 0   Response ID                                                                                                                                                           400 non-null    object 
 1   Live streaming:BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?                                   14 non-null     float64
 2   Collaboration with Influencer:BOSCH:GM - And which of these types of content have you seen recently for these 

In [4]:
Germany.head(2) #initial dataset

,Response ID,Live streaming:BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:MIELE:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:MIELE:GM - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:HAIER:GM - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):HAIER:GM - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:HAIER:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:ROWENTA:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:ROWENTA:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:ROWENTA:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:ROWENTA:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:ROWENTA:GM - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):ROWENTA:GM - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:ROWENTA:GM - And which of these types of content have you seen recently for these brands when looking at social media?
0,GER2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GER2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_GM = Germany.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_GM['SCM_Platform_cleaned'] = unpivoted_GM['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_GM = unpivoted_GM[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_GM['SCM_Platform_cleaned'] = unpivoted_GM['SCM_Platform_cleaned'].str.replace(
    ":GM - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_GM.tail())

      Response ID                                       SCM_Platform  \
33595     GER2396  None of the above:ROWENTA:GM - And which of th...   
33596     GER2397  None of the above:ROWENTA:GM - And which of th...   
33597     GER2398  None of the above:ROWENTA:GM - And which of th...   
33598     GER2399  None of the above:ROWENTA:GM - And which of th...   
33599     GER2400  None of the above:ROWENTA:GM - And which of th...   

            SCM_Platform_cleaned  Y/N  
33595  None of the above:ROWENTA  NaN  
33596  None of the above:ROWENTA  NaN  
33597  None of the above:ROWENTA  NaN  
33598  None of the above:ROWENTA  NaN  
33599  None of the above:ROWENTA  NaN  


In [6]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_GM['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_GM['Brand'] = brand_names
unpivoted_GM['Ad_Type'] = ad_types

In [7]:
count = unpivoted_GM['Y/N'].value_counts()
print(count)

Y/N
3.0     224
5.0     193
6.0     139
4.0     136
1.0     113
99.0    106
2.0      89
Name: count, dtype: int64


In [8]:
# Drop the original 'SCM_Platform' column
unpivoted_GM = unpivoted_GM.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_GM = unpivoted_GM.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_GM = unpivoted_GM[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_GM['Y/N'] = np.where(unpivoted_GM['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_GM['Y/N'] == 99, 99,  
                        np.where(unpivoted_GM['Y/N'].isna(), 0, unpivoted_GM['Y/N'])))

In [9]:
unpivoted_GM.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,GER2001,Live streaming:BOSCH:GM - And which of these t...,Live streaming,0.0,BOSCH
1,GER2002,Live streaming:BOSCH:GM - And which of these t...,Live streaming,0.0,BOSCH
2,GER2003,Live streaming:BOSCH:GM - And which of these t...,Live streaming,0.0,BOSCH
3,GER2004,Live streaming:BOSCH:GM - And which of these t...,Live streaming,0.0,BOSCH
4,GER2005,Live streaming:BOSCH:GM - And which of these t...,Live streaming,0.0,BOSCH


In [10]:
count = unpivoted_GM['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_GM['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_GM)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32600
1.0       894
99.0      106
Name: count, dtype: int64


Brand
BOSCH        2800
MIELE        2800
SIEMENS      2800
AEG          2800
SAMSUNG      2800
BAUKNECHT    2800
PHILIPS      2800
LG           2800
DYSON        2800
NEFF         2800
HAIER        2800
ROWENTA      2800
Name: count, dtype: int64


Total number of rows: 33600


### Sweden

In [11]:
Sweden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                 --------------  -----  
 0   Response ID                                                                                                                                                            400 non-null    object 
 1   Live streaming:ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?                               12 non-null     float64
 2   Collaboration with Influencer:ELECTROLUX:SW - And which of these types of content have you seen recently f

In [12]:
Sweden.head(2) #initial dataset

,Response ID,Live streaming:ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:BOSCH:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:BOSCH:SW - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:AEG:SW - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):AEG:SW - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:AEG:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:ROBOROCK:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:ROBOROCK:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:ROBOROCK:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:ROBOROCK:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:ROBOROCK:SW - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):ROBOROCK:SW - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:ROBOROCK:SW - And which of these types of content have you seen recently for these brands when looking at social media?
0,SWE2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SWE2002,1.0,NaN,3.0,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_SW = Sweden.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_SW['SCM_Platform_cleaned'] = unpivoted_SW['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_SW = unpivoted_SW[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_SW['SCM_Platform_cleaned'] = unpivoted_SW['SCM_Platform_cleaned'].str.replace(
    ":SW - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_SW.tail())

      Response ID                                       SCM_Platform  \
33595     SWE2396  None of the above:ROBOROCK:SW - And which of t...   
33596     SWE2397  None of the above:ROBOROCK:SW - And which of t...   
33597     SWE2398  None of the above:ROBOROCK:SW - And which of t...   
33598     SWE2399  None of the above:ROBOROCK:SW - And which of t...   
33599     SWE2400  None of the above:ROBOROCK:SW - And which of t...   

             SCM_Platform_cleaned  Y/N  
33595  None of the above:ROBOROCK  NaN  
33596  None of the above:ROBOROCK  NaN  
33597  None of the above:ROBOROCK  NaN  
33598  None of the above:ROBOROCK  NaN  
33599  None of the above:ROBOROCK  NaN  


In [14]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_SW['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SW['Brand'] = brand_names
unpivoted_SW['Ad_Type'] = ad_types

In [15]:
count = unpivoted_SW['Y/N'].value_counts()
print(count)

Y/N
3.0     233
5.0     185
6.0     162
4.0     128
2.0      87
1.0      76
99.0     68
Name: count, dtype: int64


In [16]:
# Drop the original 'SCM_Platform' column
unpivoted_SW = unpivoted_SW.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_SW = unpivoted_SW.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_SW = unpivoted_SW[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_SW['Y/N'] = np.where(unpivoted_SW['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_SW['Y/N'] == 99, 99,  
                        np.where(unpivoted_SW['Y/N'].isna(), 0, unpivoted_SW['Y/N'])))

In [17]:
unpivoted_SW.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,SWE2001,Live streaming:ELECTROLUX:SW - And which of th...,Live streaming,0.0,ELECTROLUX
1,SWE2002,Live streaming:ELECTROLUX:SW - And which of th...,Live streaming,1.0,ELECTROLUX
2,SWE2003,Live streaming:ELECTROLUX:SW - And which of th...,Live streaming,0.0,ELECTROLUX
3,SWE2004,Live streaming:ELECTROLUX:SW - And which of th...,Live streaming,0.0,ELECTROLUX
4,SWE2005,Live streaming:ELECTROLUX:SW - And which of th...,Live streaming,0.0,ELECTROLUX


In [18]:
count = unpivoted_SW['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_SW['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SW)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32661
1.0       871
99.0       68
Name: count, dtype: int64


Brand
ELECTROLUX    2800
BOSCH         2800
SAMSUNG       2800
MIELE         2800
PHILIPS       2800
SIEMENS       2800
CYLINDA       2800
WHIRLPOOL     2800
LG            2800
DYSON         2800
AEG           2800
ROBOROCK      2800
Name: count, dtype: int64


Total number of rows: 33600


### Italy

In [19]:
Italy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                 --------------  -----  
 0   Response ID                                                                                                                                                            400 non-null    object 
 1   Live streaming:SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?                                  16 non-null     float64
 2   Collaboration with Influencer:SAMSUNG:IT - And which of these types of content have you seen recently for 

In [20]:
Italy.head(2) #initial dataset

,Response ID,Live streaming:SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:BOSCH:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:BOSCH:IT - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:HAIER:IT - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):HAIER:IT - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:HAIER:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:SIEMENS:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SIEMENS:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SIEMENS:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SIEMENS:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SIEMENS:IT - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SIEMENS:IT - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SIEMENS:IT - And which of these types of content have you seen recently for these brands when looking at social media?
0,ITA2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ITA2002,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN


In [21]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_IT = Italy.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_IT['SCM_Platform_cleaned'] = unpivoted_IT['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_IT = unpivoted_IT[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_IT['SCM_Platform_cleaned'] = unpivoted_IT['SCM_Platform_cleaned'].str.replace(
    ":IT - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_IT.tail())

      Response ID                                       SCM_Platform  \
33595     ITA2396  None of the above:SIEMENS:IT - And which of th...   
33596     ITA2397  None of the above:SIEMENS:IT - And which of th...   
33597     ITA2398  None of the above:SIEMENS:IT - And which of th...   
33598     ITA2399  None of the above:SIEMENS:IT - And which of th...   
33599     ITA2400  None of the above:SIEMENS:IT - And which of th...   

            SCM_Platform_cleaned  Y/N  
33595  None of the above:SIEMENS  NaN  
33596  None of the above:SIEMENS  NaN  
33597  None of the above:SIEMENS  NaN  
33598  None of the above:SIEMENS  NaN  
33599  None of the above:SIEMENS  NaN  


In [22]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_IT['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_IT['Brand'] = brand_names
unpivoted_IT['Ad_Type'] = ad_types

In [23]:
count = unpivoted_IT['Y/N'].value_counts()
print(count)

Y/N
3.0     257
5.0     235
6.0     177
4.0     172
1.0     145
2.0     105
99.0     77
Name: count, dtype: int64


In [24]:
# Drop the original 'SCM_Platform' column
unpivoted_IT = unpivoted_IT.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_IT = unpivoted_IT.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_IT = unpivoted_IT[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_IT['Y/N'] = np.where(unpivoted_IT['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_IT['Y/N'] == 99, 99,  
                        np.where(unpivoted_IT['Y/N'].isna(), 0, unpivoted_IT['Y/N'])))

In [25]:
unpivoted_IT.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,ITA2001,Live streaming:SAMSUNG:IT - And which of these...,Live streaming,0.0,SAMSUNG
1,ITA2002,Live streaming:SAMSUNG:IT - And which of these...,Live streaming,0.0,SAMSUNG
2,ITA2003,Live streaming:SAMSUNG:IT - And which of these...,Live streaming,0.0,SAMSUNG
3,ITA2004,Live streaming:SAMSUNG:IT - And which of these...,Live streaming,0.0,SAMSUNG
4,ITA2005,Live streaming:SAMSUNG:IT - And which of these...,Live streaming,0.0,SAMSUNG


In [26]:
count = unpivoted_IT['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_IT['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_IT)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32432
1.0      1091
99.0       77
Name: count, dtype: int64


Brand
SAMSUNG       2800
BOSCH         2800
WHIRLPOOL     2800
LG            2800
DYSON         2800
MIELE         2800
ELECTROLUX    2800
HOTPOINT      2800
BEKO          2800
AEG           2800
HAIER         2800
SIEMENS       2800
Name: count, dtype: int64


Total number of rows: 33600


### UK

In [27]:
UK.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                               Non-Null Count  Dtype  
---  ------                                                                                                                                                               --------------  -----  
 0   Response ID                                                                                                                                                          400 non-null    object 
 1   Live streaming:SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?                                19 non-null     float64
 2   Collaboration with Influencer:SAMSUNG:UK - And which of these types of content have you seen recently for these br

In [28]:
UK.head(2) #initial dataset

,Response ID,Live streaming:SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:LG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:LG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:SIEMENS:UK - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SIEMENS:UK - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SIEMENS:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:NEFF:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:NEFF:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:NEFF:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:NEFF:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:NEFF:UK - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):NEFF:UK - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:NEFF:UK - And which of these types of content have you seen recently for these brands when looking at social media?
0,UK2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UK2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_UK = UK.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_UK['SCM_Platform_cleaned'] = unpivoted_UK['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_UK = unpivoted_UK[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_UK['SCM_Platform_cleaned'] = unpivoted_UK['SCM_Platform_cleaned'].str.replace(
    ":UK - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_UK.tail())

      Response ID                                       SCM_Platform  \
33595      UK2396  None of the above:NEFF:UK - And which of these...   
33596      UK2397  None of the above:NEFF:UK - And which of these...   
33597      UK2398  None of the above:NEFF:UK - And which of these...   
33598      UK2399  None of the above:NEFF:UK - And which of these...   
33599      UK2400  None of the above:NEFF:UK - And which of these...   

         SCM_Platform_cleaned  Y/N  
33595  None of the above:NEFF  NaN  
33596  None of the above:NEFF  NaN  
33597  None of the above:NEFF  NaN  
33598  None of the above:NEFF  NaN  
33599  None of the above:NEFF  NaN  


In [30]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_UK['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_UK['Brand'] = brand_names
unpivoted_UK['Ad_Type'] = ad_types

In [31]:
count = unpivoted_UK['Y/N'].value_counts()
print(count)

Y/N
3.0     265
5.0     222
6.0     182
4.0     181
1.0     140
2.0     137
99.0     50
Name: count, dtype: int64


In [32]:
# Drop the original 'SCM_Platform' column
unpivoted_UK = unpivoted_UK.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_UK = unpivoted_UK.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_UK = unpivoted_UK[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_UK['Y/N'] = np.where(unpivoted_UK['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_UK['Y/N'] == 99, 99,  
                        np.where(unpivoted_UK['Y/N'].isna(), 0, unpivoted_UK['Y/N'])))

In [33]:
unpivoted_UK.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,UK2001,Live streaming:SAMSUNG:UK - And which of these...,Live streaming,0.0,SAMSUNG
1,UK2002,Live streaming:SAMSUNG:UK - And which of these...,Live streaming,0.0,SAMSUNG
2,UK2003,Live streaming:SAMSUNG:UK - And which of these...,Live streaming,0.0,SAMSUNG
3,UK2004,Live streaming:SAMSUNG:UK - And which of these...,Live streaming,0.0,SAMSUNG
4,UK2005,Live streaming:SAMSUNG:UK - And which of these...,Live streaming,0.0,SAMSUNG


In [34]:
count = unpivoted_UK['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_UK['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_UK)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32423
1.0      1127
99.0       50
Name: count, dtype: int64


Brand
SAMSUNG     2800
LG          2800
HOTPOINT    2800
BOSCH       2800
DYSON       2800
BEKO        2800
SHARK       2800
ZANUSSI     2800
MIELE       2800
AEG         2800
SIEMENS     2800
NEFF        2800
Name: count, dtype: int64


Total number of rows: 33600


### Poland

In [35]:
Poland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                 --------------  -----  
 0   Response ID                                                                                                                                                            400 non-null    object 
 1   Live streaming:SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?                                  18 non-null     float64
 2   Collaboration with Influencer:SAMSUNG:PO - And which of these types of content have you seen recently for 

In [36]:
Poland.head(2) #initial dataset

,Response ID,Live streaming:SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:BOSCH:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:BOSCH:PO - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:MIELE:PO - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):MIELE:PO - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:MIELE:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:AEG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:AEG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:AEG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:AEG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:AEG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):AEG:PO - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:AEG:PO - And which of these types of content have you seen recently for these brands when looking at social media?
0,POL2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POL2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_PO = Poland.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_PO['SCM_Platform_cleaned'] = unpivoted_PO['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_PO = unpivoted_PO[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_PO['SCM_Platform_cleaned'] = unpivoted_PO['SCM_Platform_cleaned'].str.replace(
    ":PO - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_PO.tail())

      Response ID                                       SCM_Platform  \
33595     POL2396  None of the above:AEG:PO - And which of these ...   
33596     POL2397  None of the above:AEG:PO - And which of these ...   
33597     POL2398  None of the above:AEG:PO - And which of these ...   
33598     POL2399  None of the above:AEG:PO - And which of these ...   
33599     POL2400  None of the above:AEG:PO - And which of these ...   

        SCM_Platform_cleaned  Y/N  
33595  None of the above:AEG  NaN  
33596  None of the above:AEG  NaN  
33597  None of the above:AEG  NaN  
33598  None of the above:AEG  NaN  
33599  None of the above:AEG  NaN  


In [38]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_PO['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_PO['Brand'] = brand_names
unpivoted_PO['Ad_Type'] = ad_types

In [39]:
count = unpivoted_PO['Y/N'].value_counts()
print(count)

Y/N
3.0     274
5.0     191
6.0     182
4.0     122
2.0     106
1.0     103
99.0     65
Name: count, dtype: int64


In [40]:
# Drop the original 'SCM_Platform' column
unpivoted_PO = unpivoted_PO.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_PO = unpivoted_PO.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_PO = unpivoted_PO[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_PO['Y/N'] = np.where(unpivoted_PO['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_PO['Y/N'] == 99, 99,  
                        np.where(unpivoted_PO['Y/N'].isna(), 0, unpivoted_PO['Y/N'])))

In [41]:
unpivoted_PO.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,POL2001,Live streaming:SAMSUNG:PO - And which of these...,Live streaming,0.0,SAMSUNG
1,POL2002,Live streaming:SAMSUNG:PO - And which of these...,Live streaming,0.0,SAMSUNG
2,POL2003,Live streaming:SAMSUNG:PO - And which of these...,Live streaming,0.0,SAMSUNG
3,POL2004,Live streaming:SAMSUNG:PO - And which of these...,Live streaming,1.0,SAMSUNG
4,POL2005,Live streaming:SAMSUNG:PO - And which of these...,Live streaming,0.0,SAMSUNG


In [42]:
count = unpivoted_PO['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_PO['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_PO)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32557
1.0       978
99.0       65
Name: count, dtype: int64


Brand
SAMSUNG       2800
BOSCH         2800
ELECTROLUX    2800
BEKO          2800
AMICA         2800
PHILIPS       2800
WHIRLPOOL     2800
LG            2800
DYSON         2800
HAIER         2800
MIELE         2800
AEG           2800
Name: count, dtype: int64


Total number of rows: 33600


### Thailand

In [43]:
Thailand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                 --------------  -----  
 0   Response ID                                                                                                                                                            400 non-null    object 
 1   Live streaming:SAMSUNG:TH - And which of these types of content have you seen recently for these brands when looking at social media?                                  58 non-null     float64
 2   Collaboration with Influencer:SAMSUNG:TH - And which of these types of content have you seen recently for 

In [44]:
Thailand.head(2) #initial dataset

,Response ID,Live streaming:SAMSUNG:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SAMSUNG:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SAMSUNG:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SAMSUNG:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SAMSUNG:TH - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SAMSUNG:TH - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SAMSUNG:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:LG:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:LG:TH - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:BOSCH:TH - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):BOSCH:TH - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:BOSCH:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:DYSON:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:DYSON:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:DYSON:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:DYSON:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:DYSON:TH - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):DYSON:TH - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:DYSON:TH - And which of these types of content have you seen recently for these brands when looking at social media?
0,TH2001,NaN,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,3.0,NaN,NaN,NaN,NaN
1,TH2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_TH = Thailand.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_TH['SCM_Platform_cleaned'] = unpivoted_TH['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_TH = unpivoted_TH[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_TH['SCM_Platform_cleaned'] = unpivoted_TH['SCM_Platform_cleaned'].str.replace(
    ":TH - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_TH.tail())

      Response ID                                       SCM_Platform  \
33595      TH2396  None of the above:DYSON:TH - And which of thes...   
33596      TH2397  None of the above:DYSON:TH - And which of thes...   
33597      TH2398  None of the above:DYSON:TH - And which of thes...   
33598      TH2399  None of the above:DYSON:TH - And which of thes...   
33599      TH2400  None of the above:DYSON:TH - And which of thes...   

          SCM_Platform_cleaned  Y/N  
33595  None of the above:DYSON  NaN  
33596  None of the above:DYSON  NaN  
33597  None of the above:DYSON  NaN  
33598  None of the above:DYSON  NaN  
33599  None of the above:DYSON  NaN  


In [46]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_TH['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_TH['Brand'] = brand_names
unpivoted_TH['Ad_Type'] = ad_types

In [47]:
count = unpivoted_TH['Y/N'].value_counts()
print(count)

Y/N
3.0     1032
4.0      785
6.0      672
2.0      542
5.0      522
1.0      478
99.0      59
Name: count, dtype: int64


In [48]:
# Drop the original 'SCM_Platform' column
unpivoted_TH = unpivoted_TH.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_TH = unpivoted_TH.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_TH = unpivoted_TH[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_TH['Y/N'] = np.where(unpivoted_TH['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_TH['Y/N'] == 99, 99,  
                        np.where(unpivoted_TH['Y/N'].isna(), 0, unpivoted_TH['Y/N'])))

In [49]:
unpivoted_TH.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,TH2001,Live streaming:SAMSUNG:TH - And which of these...,Live streaming,0.0,SAMSUNG
1,TH2002,Live streaming:SAMSUNG:TH - And which of these...,Live streaming,0.0,SAMSUNG
2,TH2003,Live streaming:SAMSUNG:TH - And which of these...,Live streaming,0.0,SAMSUNG
3,TH2004,Live streaming:SAMSUNG:TH - And which of these...,Live streaming,0.0,SAMSUNG
4,TH2005,Live streaming:SAMSUNG:TH - And which of these...,Live streaming,0.0,SAMSUNG


In [50]:
count = unpivoted_TH['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_TH['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_TH)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     29510
1.0      4031
99.0       59
Name: count, dtype: int64


Brand
SAMSUNG       2800
LG            2800
MITSUBISHI    2800
ELECTROLUX    2800
PANASONIC     2800
SHARP         2800
PHILIPS       2800
HITACHI       2800
HAIER         2800
TEFAL         2800
BOSCH         2800
DYSON         2800
Name: count, dtype: int64


Total number of rows: 33600


### Vietnam

In [51]:
Vietnam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                 --------------  -----  
 0   Response ID                                                                                                                                                            400 non-null    object 
 1   Live streaming:SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?                                  61 non-null     float64
 2   Collaboration with Influencer:SAMSUNG:VN - And which of these types of content have you seen recently for 

In [52]:
Vietnam.head(2) #initial dataset

,Response ID,Live streaming:SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:PANASONIC:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:PANASONIC:VN - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:BOSCH:VN - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):BOSCH:VN - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:BOSCH:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:MIDEA:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:MIDEA:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:MIDEA:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:MIDEA:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:MIDEA:VN - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):MIDEA:VN - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:MIDEA:VN - And which of these types of content have you seen recently for these brands when looking at social media?
0,VN02001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VN02002,NaN,2.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_VN = Vietnam.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_VN['SCM_Platform_cleaned'] = unpivoted_VN['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_VN = unpivoted_VN[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_VN['SCM_Platform_cleaned'] = unpivoted_VN['SCM_Platform_cleaned'].str.replace(
    ":VN - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_VN.tail())

      Response ID                                       SCM_Platform  \
33595     VN02396  None of the above:MIDEA:VN - And which of thes...   
33596     VN02397  None of the above:MIDEA:VN - And which of thes...   
33597     VN02398  None of the above:MIDEA:VN - And which of thes...   
33598     VN02399  None of the above:MIDEA:VN - And which of thes...   
33599     VN02400  None of the above:MIDEA:VN - And which of thes...   

          SCM_Platform_cleaned  Y/N  
33595  None of the above:MIDEA  NaN  
33596  None of the above:MIDEA  NaN  
33597  None of the above:MIDEA  NaN  
33598  None of the above:MIDEA  NaN  
33599  None of the above:MIDEA  NaN  


In [54]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_VN['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_VN['Brand'] = brand_names
unpivoted_VN['Ad_Type'] = ad_types

In [55]:
count = unpivoted_VN['Y/N'].value_counts()
print(count)

Y/N
3.0     1006
4.0      774
6.0      718
5.0      542
2.0      520
1.0      505
99.0      30
Name: count, dtype: int64


In [56]:
# Drop the original 'SCM_Platform' column
unpivoted_VN = unpivoted_VN.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_VN = unpivoted_VN.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_VN = unpivoted_VN[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_VN['Y/N'] = np.where(unpivoted_VN['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_VN['Y/N'] == 99, 99,  
                        np.where(unpivoted_VN['Y/N'].isna(), 0, unpivoted_VN['Y/N'])))

In [57]:
unpivoted_VN.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,VN02001,Live streaming:SAMSUNG:VN - And which of these...,Live streaming,0.0,SAMSUNG
1,VN02002,Live streaming:SAMSUNG:VN - And which of these...,Live streaming,0.0,SAMSUNG
2,VN02003,Live streaming:SAMSUNG:VN - And which of these...,Live streaming,0.0,SAMSUNG
3,VN02004,Live streaming:SAMSUNG:VN - And which of these...,Live streaming,0.0,SAMSUNG
4,VN02005,Live streaming:SAMSUNG:VN - And which of these...,Live streaming,0.0,SAMSUNG


In [58]:
count = unpivoted_VN['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_VN['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_VN)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     29505
1.0      4065
99.0       30
Name: count, dtype: int64


Brand
SAMSUNG       2800
PANASONIC     2800
LG            2800
TOSHIBA       2800
ELECTROLUX    2800
SHARP         2800
PHILIPS       2800
HITACHI       2800
XIAOMI        2800
AQUA          2800
BOSCH         2800
MIDEA         2800
Name: count, dtype: int64


Total number of rows: 33600


### Australia

In [59]:
Australia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                                    Non-Null Count  Dtype  
---  ------                                                                                                                                                                    --------------  -----  
 0   Response ID                                                                                                                                                               400 non-null    object 
 1   Live streaming:SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?                                     51 non-null     float64
 2   Collaboration with Influencer:SAMSUNG:AU - And which of these types of content have you seen r

In [60]:
Australia.head(2) #initial dataset

,Response ID,Live streaming:SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:LG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:LG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:HAIER:AU - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):HAIER:AU - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:HAIER:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:AEG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:AEG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:AEG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:AEG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:AEG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):AEG:AU - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:AEG:AU - And which of these types of content have you seen recently for these brands when looking at social media?
0,AUS2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AUS2002,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_AU = Australia.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_AU['SCM_Platform_cleaned'] = unpivoted_AU['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_AU = unpivoted_AU[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_AU['SCM_Platform_cleaned'] = unpivoted_AU['SCM_Platform_cleaned'].str.replace(
    ":AU - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_AU.tail())

      Response ID                                       SCM_Platform  \
33595     AUS2396  None of the above:AEG:AU - And which of these ...   
33596     AUS2397  None of the above:AEG:AU - And which of these ...   
33597     AUS2398  None of the above:AEG:AU - And which of these ...   
33598     AUS2399  None of the above:AEG:AU - And which of these ...   
33599     AUS2400  None of the above:AEG:AU - And which of these ...   

        SCM_Platform_cleaned  Y/N  
33595  None of the above:AEG  NaN  
33596  None of the above:AEG  NaN  
33597  None of the above:AEG  NaN  
33598  None of the above:AEG  NaN  
33599  None of the above:AEG  NaN  


In [62]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_AU['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_AU['Brand'] = brand_names
unpivoted_AU['Ad_Type'] = ad_types

In [63]:
count = unpivoted_AU['Y/N'].value_counts()
print(count)

Y/N
3.0     787
4.0     626
6.0     539
5.0     440
1.0     404
2.0     349
99.0     71
Name: count, dtype: int64


In [64]:
# Drop the original 'SCM_Platform' column
unpivoted_AU = unpivoted_AU.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_AU = unpivoted_AU.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_AU = unpivoted_AU[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_AU['Y/N'] = np.where(unpivoted_AU['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_AU['Y/N'] == 99, 99,  
                        np.where(unpivoted_AU['Y/N'].isna(), 0, unpivoted_AU['Y/N'])))

In [65]:
unpivoted_AU.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,AUS2001,Live streaming:SAMSUNG:AU - And which of these...,Live streaming,0.0,SAMSUNG
1,AUS2002,Live streaming:SAMSUNG:AU - And which of these...,Live streaming,0.0,SAMSUNG
2,AUS2003,Live streaming:SAMSUNG:AU - And which of these...,Live streaming,0.0,SAMSUNG
3,AUS2004,Live streaming:SAMSUNG:AU - And which of these...,Live streaming,0.0,SAMSUNG
4,AUS2005,Live streaming:SAMSUNG:AU - And which of these...,Live streaming,1.0,SAMSUNG


In [66]:
count = unpivoted_AU['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     30384
1.0      3145
99.0       71
Name: count, dtype: int64


Brand
SAMSUNG         2800
LG              2800
DYSON           2800
WESTINGHOUSE    2800
BOSCH           2800
HISENSE         2800
MIELE           2800
SMEG            2800
ELECTROLUX      2800
HAIER           2800
AEG             2800
Name: count, dtype: int64


Total number of rows: 33600


In [67]:
# Count blank (empty or NaN) values in Brand
blank_count = unpivoted_AU[unpivoted_AU['Brand'].isna() | (unpivoted_AU['Brand'] == '')].shape[0]
print("Count of blank values in Brand:", blank_count)

Count of blank values in Brand: 2800


In [68]:
# Replace NaN values in 'Brand' column with 'FISHER&PAYKEL'
unpivoted_AU['Brand'] = unpivoted_AU['Brand'].fillna('FISHER&PAYKEL')

# Check the count of NaN values after replacement
na_count = unpivoted_AU['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [69]:
count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Brand
SAMSUNG          2800
LG               2800
DYSON            2800
WESTINGHOUSE     2800
FISHER&PAYKEL    2800
BOSCH            2800
HISENSE          2800
MIELE            2800
SMEG             2800
ELECTROLUX       2800
HAIER            2800
AEG              2800
Name: count, dtype: int64


Total number of rows: 33600


### Switzerland

In [70]:
Switzerland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                 --------------  -----  
 0   Response ID                                                                                                                                                            400 non-null    object 
 1   Live streaming:V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?                                    10 non-null     float64
 2   Collaboration with Influencer:V ZUG:SZ - And which of these types of content have you seen recently for th

In [71]:
Switzerland.head(2) #initial dataset

,Response ID,Live streaming:V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:ELECTROLUX:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:ELECTROLUX:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:LIEBHERR:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):LIEBHERR:SZ - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:LIEBHERR:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:SCHULTHESS:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SCHULTHESS:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SCHULTHESS:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SCHULTHESS:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SCHULTHESS:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SCHULTHESS:SZ - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SCHULTHESS:SZ - And which of these types of content have you seen recently for these brands when looking at social media?
0,SWIT2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SWIT2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_SZ = Switzerland.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_SZ['SCM_Platform_cleaned'] = unpivoted_SZ['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_SZ['SCM_Platform_cleaned'] = unpivoted_SZ['SCM_Platform_cleaned'].str.replace(
    ":SZ - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_SZ.tail())

      Response ID                                       SCM_Platform  \
33595    SWIT2396  None of the above:SCHULTHESS:SZ - And which of...   
33596    SWIT2397  None of the above:SCHULTHESS:SZ - And which of...   
33597    SWIT2398  None of the above:SCHULTHESS:SZ - And which of...   
33598    SWIT2399  None of the above:SCHULTHESS:SZ - And which of...   
33599    SWIT2400  None of the above:SCHULTHESS:SZ - And which of...   

               SCM_Platform_cleaned  Y/N  
33595  None of the above:SCHULTHESS  NaN  
33596  None of the above:SCHULTHESS  NaN  
33597  None of the above:SCHULTHESS  NaN  
33598  None of the above:SCHULTHESS  NaN  
33599  None of the above:SCHULTHESS  NaN  


In [73]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_SZ['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SZ['Brand'] = brand_names
unpivoted_SZ['Ad_Type'] = ad_types

In [74]:
count = unpivoted_SZ['Y/N'].value_counts()
print(count)

Y/N
3.0     244
5.0     175
6.0     109
4.0     101
1.0      87
2.0      62
99.0     39
Name: count, dtype: int64


In [75]:
# Drop the original 'SCM_Platform' column
unpivoted_SZ = unpivoted_SZ.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_SZ = unpivoted_SZ.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_SZ['Y/N'] = np.where(unpivoted_SZ['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_SZ['Y/N'] == 99, 99,  
                        np.where(unpivoted_SZ['Y/N'].isna(), 0, unpivoted_SZ['Y/N'])))

In [76]:
unpivoted_SZ.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,SWIT2001,Live streaming:V ZUG:SZ - And which of these t...,Live streaming,0.0,None
1,SWIT2002,Live streaming:V ZUG:SZ - And which of these t...,Live streaming,0.0,None
2,SWIT2003,Live streaming:V ZUG:SZ - And which of these t...,Live streaming,0.0,None
3,SWIT2004,Live streaming:V ZUG:SZ - And which of these t...,Live streaming,0.0,None
4,SWIT2005,Live streaming:V ZUG:SZ - And which of these t...,Live streaming,0.0,None


In [77]:
count = unpivoted_SZ['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32783
1.0       778
99.0       39
Name: count, dtype: int64


Brand
ELECTROLUX    2800
MIELE         2800
DYSON         2800
BOSCH         2800
SAMSUNG       2800
SIEMENS       2800
PHILIPS       2800
BAUKNECHT     2800
AEG           2800
LIEBHERR      2800
SCHULTHESS    2800
Name: count, dtype: int64


Total number of rows: 33600


In [78]:
# Count blank (empty or NaN) values in Brand
blank_count = unpivoted_SZ[unpivoted_SZ['Brand'].isna() | (unpivoted_SZ['Brand'] == '')].shape[0]
print("Count of blank values in 'Brand:", blank_count)

Count of blank values in 'Brand: 2800


In [79]:
# Replace NaN values in 'Brand' column with 'V Zug'
unpivoted_SZ['Brand'] = unpivoted_SZ['Brand'].fillna('V ZUG')

# Check the count of NaN values after replacement
na_count = unpivoted_SZ['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [80]:
count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Brand
V ZUG         2800
ELECTROLUX    2800
MIELE         2800
DYSON         2800
BOSCH         2800
SAMSUNG       2800
SIEMENS       2800
PHILIPS       2800
BAUKNECHT     2800
AEG           2800
LIEBHERR      2800
SCHULTHESS    2800
Name: count, dtype: int64


Total number of rows: 33600


### Concat and save them in another csv file

In [81]:
# Concatenate the melted DataFrames
combined_q16 = pd.concat([unpivoted_GM, unpivoted_SW, unpivoted_TH, unpivoted_IT, unpivoted_PO, 
                                unpivoted_SZ, unpivoted_VN, unpivoted_AU, unpivoted_UK], ignore_index=True)

# combined_q16.dropna(subset=['Response ID'], inplace=True)

combined_q16 = combined_q16.sort_values(by='Response ID', ascending=True)

# combined_q16.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\SEPTEMBER\combined_q16.xlsx', float_format='%.0f', index=False)
# combined_q16.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\OCTOBER\combined_q16.xlsx', float_format='%.0f', index=False)
# combined_q16.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\NOVEMBER\combined_q16.xlsx', float_format='%.0f', index=False)
# combined_q16.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\DECEMBER\combined_q16.xlsx', float_format='%.0f', index=False)
# combined_q16.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\2025\JANUARY\combined_q16.xlsx', float_format='%.0f', index=False)
combined_q16.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\2025\JULY\combined_q16.xlsx', float_format='%.0f', index=False)

print(combined_q16.head(2))

       Response ID                                       SCM_Platform  \
235200     AUS2001  Live streaming:SAMSUNG:AU - And which of these...   
263600     AUS2001  Collaboration with Influencer:HAIER:AU - And w...   

                 SCM_Platform_cleaned  Y/N    Brand  
235200                 Live streaming  0.0  SAMSUNG  
263600  Collaboration with Influencer  0.0    HAIER  


In [82]:
# Count blank (NaN) rows in the 'Response ID' column
blank_count = combined_q16['Response ID'].isna().sum()
print(f"Number of blank rows in 'Response ID': {blank_count}")

Number of blank rows in 'Response ID': 0


In [83]:
combined_q16.info()

<class 'pandas.core.frame.DataFrame'>
Index: 302400 entries, 235200 to 218399
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Response ID           302400 non-null  object 
 1   SCM_Platform          302400 non-null  object 
 2   SCM_Platform_cleaned  302400 non-null  object 
 3   Y/N                   302400 non-null  float64
 4   Brand                 302400 non-null  object 
dtypes: float64(1), object(4)
memory usage: 13.8+ MB


In [84]:
count = combined_q16['Brand'].value_counts()
print(count)

Brand
SAMSUNG          25200
BOSCH            25200
DYSON            22400
LG               22400
MIELE            19600
ELECTROLUX       19600
AEG              19600
PHILIPS          16800
HAIER            14000
SIEMENS          14000
BEKO              8400
WHIRLPOOL         8400
BAUKNECHT         5600
HITACHI           5600
NEFF              5600
HOTPOINT          5600
SHARP             5600
PANASONIC         5600
HISENSE           2800
ROWENTA           2800
SMEG              2800
AMICA             2800
WESTINGHOUSE      2800
FISHER&PAYKEL     2800
ROBOROCK          2800
V ZUG             2800
LIEBHERR          2800
CYLINDA           2800
SCHULTHESS        2800
MITSUBISHI        2800
TEFAL             2800
ZANUSSI           2800
SHARK             2800
TOSHIBA           2800
XIAOMI            2800
AQUA              2800
MIDEA             2800
Name: count, dtype: int64
